In [1]:
# # header
import sys
sys.path.append(r"../")

%load_ext autoreload
%autoreload 2

In [2]:
# # built-in modules
import os
from pprint import pformat
from collections import OrderedDict
# # Torch modules
import torch
from torch.utils.data import random_split
from torchvision import transforms, datasets
# # internal imports
from prelude import startup_folders, get_device, load_dicts
from src.composer import Recognition_DS
from src.model import AttentionModel
from src.utils import plot_all
from src.utils import build_loaders
from src.conductor import AttentionTrain


In [3]:
start_folder = r"/Users/saeedida/GitProjects/bio-attention/pretrained/mnist"
results_folder, logger = startup_folders(start_folder, name=f"exp_a")
data_path = r"../data"

/Users/saeedida/GitProjects/bio-attention/pretrained/mnist/1728902926 was created!


In [4]:
model_params = load_dicts(start_folder, "model_params")
tasks = load_dicts(start_folder, "tasks")
train_params = load_dicts(start_folder, "train_params")
DeVice, num_workers, pin_memory = get_device()
print(f"model_params: {model_params}")
print(f"tasks: {tasks}")
print(f"train_params: {train_params}")
tasks['Recognition']

Device set to mps
model_params: {'in_dims': [3, 96, 96], 'n_classes': 10, 'out_dim': 20, 'normalize': True, 'softness': 0.5, 'channels': [3, 16, 32, 64, 128, 128], 'residuals': False, 'kernels': 3, 'strides': 1, 'paddings': 'same', 'conv_bias': True, 'conv_norms': [None, 'layer', 'layer', 'layer', 'layer'], 'conv_dropouts': 0.0, 'conv_funs': ReLU(), 'deconv_funs': Tanh(), 'deconv_norms': [None, 'layer', 'layer', 'layer', 'layer'], 'pools': [2, 2, 2, 2, 3], 'rnn_dims': [128, 64], 'rnn_bias': True, 'rnn_dropouts': 0.0, 'rnn_funs': ReLU(), 'n_tasks': 7, 'task_weight': True, 'task_bias': True, 'task_funs': Tanh(), 'norm_mean': [0.5, 0.5, 0.5], 'norm_std': [1.0, 1.0, 1.0], 'rnn_to_fc': False, 'trans_fun': ReLU(), 'affine': False}
tasks: {'IOR': {'composer': 'IOR_DS', 'key': 0, 'params': {'n_digits': 3, 'n_attend': 2, 'noise': 0.25, 'overlap': 1.0}, 'datasets': ['IOR_DS', 'IOR_DS', 'IOR_DS'], 'dataloaders': [None, None, None], 'loss_w': [1.0, 1.0, 0.0], 'loss_s': [None, None], 'has_prompt': 

{'composer': 'Recognition_DS',
 'key': 4,
 'params': {'n_iter': 3,
  'stride': 16,
  'blank': False,
  'static': False,
  'noise': 0.25},
 'datasets': ['Recognition_DS', 'Recognition_DS', 'Recognition_DS'],
 'dataloaders': [None, None, None],
 'loss_w': [1.0, 0.0, 2.0],
 'loss_s': '(slice(1, None, None), None)',
 'has_prompt': False}

In [5]:
tasks = OrderedDict({})
tasks["Rec_VisDyn"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": False, "static": False, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}
tasks["Rec_VisStat"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": False, "static": True, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}
tasks["Rec_inVisDyn"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": True, "static": False, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}
tasks["Rec_inVisStat"] = {
    "composer": Recognition_DS,
    "key": 4,
    "params": {"n_iter": 3, "stride": 16, "blank": True, "static": True, "noise": 0.25},
    "datasets": [],
    "dataloaders": [],
    "loss_w": (1.0, 0.0, 2.0),  # labels, masks, last label
    "loss_s": (slice(1, None), None),  # labels, masks
    "has_prompt": False,
}

In [6]:
# datasets and dataloaders
tralid_ds = datasets.MNIST(root=data_path, train=True, download=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST(root=data_path, train=False, download=True, transform=transforms.ToTensor())
train_ds, valid_ds = random_split(tralid_ds, (50000, 10000))
DeVice, num_workers, pin_memory = get_device()
for o in tasks:
    tasks[o]["datasets"].append(tasks[o]["composer"](train_ds, **tasks[o]["params"]))
    tasks[o]["datasets"].append(tasks[o]["composer"](valid_ds, **tasks[o]["params"]))
    tasks[o]["datasets"].append(tasks[o]["composer"](test_ds, **tasks[o]["params"]))
    tasks[o]["datasets"][1].build_valid_test()
    tasks[o]["datasets"][2].build_valid_test()
    tasks[o]["dataloaders"] = build_loaders(tasks[o]["datasets"], batch_size=train_params["batch_size"], num_workers=num_workers, pin_memory=pin_memory)

Device set to mps


In [7]:
# create a blank model
model = AttentionModel(**model_params)
conductor = AttentionTrain(model, None, None, tasks, logger, results_folder)

# load states into the model
model_dir = os.path.join(start_folder, "model" + ".pth")
assert os.path.exists(model_dir), "Could not find the model.pth in the given dir!"
model.load_state_dict(torch.load(model_dir, map_location=DeVice))

<All keys matched successfully>

In [8]:
# plotting...
plot_all(10, model, tasks, results_folder, "_test", DeVice, logger, False, "test")

In [9]:
# evaluating...
conductor.eval(DeVice, "test", False)

testing...
  Task Rec_VisDyn:
    CEi Loss: 0.079243    CEe Loss: 0.030167    Pix Err: 0.023421    Att Acc: 0.526366    Cls Acc: 9909/10000
  Task Rec_VisStat:
    CEi Loss: 0.189388    CEe Loss: 0.091141    Pix Err: 0.023542    Att Acc: 0.533366    Cls Acc: 9716/10000
  Task Rec_inVisDyn:
    CEi Loss: 0.305977    CEe Loss: 0.118218    Pix Err: 0.026664    Att Acc: 0.524832    Cls Acc: 9665/10000
  Task Rec_inVisStat:
    CEi Loss: 1.162786    CEe Loss: 0.642558    Pix Err: 0.026286    Att Acc: 0.527490    Cls Acc: 8490/10000
